In [ ]:
%pip install requests

In [23]:
import requests
import json
import os

In [24]:
#Define path
SECRETS_FILE = '/Users/gmccurdy/repos/curriculum/scripts/utilities/secrets.json'

# Load secrets
with open(SECRETS_FILE, 'r') as file:
    secrets = json.load(file)

# Assign variables from the secrets
API_URL = secrets.get('API_URL')
API_KEY = secrets.get('API_KEY')
COURSE_ID = secrets.get('COURSE_ID_STATISTICS')

#Define headers
headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

In [ ]:
# Create the quiz
quiz_title = input('Quiz Title')
quiz_description = input('Quiz Description')
quiz_data = {
    'quiz': {
        'title': quiz_title,
        'description': f'<p>{quiz_description}</p>',
        'quiz_type': 'assignment',
        'published': False
    }
}

create_quiz_url = f'{API_URL}/api/v1/courses/{COURSE_ID}/quizzes'
response = requests.post(create_quiz_url, headers=headers, json=quiz_data)

if response.status_code == 200:
    quiz = response.json()
    quiz_id = quiz['id']
    print(f'Quiz created successfully with ID: {quiz_id}')
else:
    print('Failed to create quiz')
    print(f"Status Code: {response.status_code}")
    print(response.text)

In [26]:
multiple_choice_questions = [
    {
        'question_name': 'Expected number of heads in 10 flips of a fair coin',
        'question_type': 'multiple_choice_question',
        'question_text': '''You flip a fair coin 10 times. What is the **expected number of heads**?''',
        'points_possible': 1,
        'answers': [
            {'text': '5', 'weight': 100},  # Correct answer
            {'text': '10', 'weight': 0},
            {'text': '2.5', 'weight': 0},
            {'text': '7', 'weight': 0}
        ]
    },
    {
        'question_name': 'Expected number of defective items in 40 parts',
        'question_type': 'multiple_choice_question',
        'question_text': '''A vending machine dispenses 90% working items. What is the **expected number of defective items** in a batch of 40 parts?''',
        'points_possible': 1,
        'answers': [
            {'text': '4', 'weight': 100},  # Correct answer
            {'text': '36', 'weight': 0},
            {'text': '6', 'weight': 0},
            {'text': '8', 'weight': 0}
        ]
    },
    {
        'question_name': 'Expected number of wins in 20 games',
        'question_type': 'multiple_choice_question',
        'question_text': '''If the probability of winning a game is 0.6, what is the **expected number of wins** in 20 games?''',
        'points_possible': 1,
        'answers': [
            {'text': '12', 'weight': 100},  # Correct answer
            {'text': '8', 'weight': 0},
            {'text': '10', 'weight': 0},
            {'text': '14', 'weight': 0}
        ]
    }
]

In [27]:
essay_questions = [
  {
    'question_name': 'Probability of Exactly 7 Heads',
    'question_type': 'essay_question',
    'question_text': '''A coin with a probability of heads being 0.7 is flipped 10 times. Calculate the probability of getting exactly 7 heads. Show all your work and provide your answer rounded to four decimal places.''',
    'points_possible': 1
  }
]

In [28]:
numerical_questions = [
    {
        'question_name': 'Expected number of free throws until first success',
        'question_type': 'numerical_question',
        'question_text': '''A basketball player has a 70% free-throw success rate. What is the **expected number of free throws until the first success**? Round your answer to 4 decimal places.''',
        'points_possible': 1,
        'answers': [
            {
                'numerical_answer_type': 'exact_answer',
                'answer_exact': '1.4286'  # Correct answer rounded to 4 decimal places
            }
        ]
    },
    {
        'question_name': 'Expected number of sixes in 5 rolls of a biased die',
        'question_type': 'numerical_question',
        'question_text': '''A biased die has a 1/3 probability of showing a 6. What is the **expected number of sixes in 5 rolls**? Round your answer to 4 decimal places.''',
        'points_possible': 1,
        'answers': [
            {
                'numerical_answer_type': 'exact_answer',
                'answer_exact': '1.6667'  # Correct answer rounded to 4 decimal places
            }
        ]
    },
    {
        'question_name': 'Expected total gain after 10 flips in a coin game',
        'question_type': 'numerical_question',
        'question_text': '''A game offers \$5 for heads and costs \$2 for tails. What is your **expected total gain after 10 flips**?''',
        'points_possible': 1,
        'answers': [
            {
                'numerical_answer_type': 'exact_answer',
                'answer_exact': '15'  # Correct answer
            }
        ]
    }
]

In [ ]:
# Combine all questions, including the essay question
questions = multiple_choice_questions + numerical_questions + essay_questions

# Add questions to the quiz
for question in questions:
    create_question_url = f'{API_URL}/api/v1/courses/{COURSE_ID}/quizzes/{quiz_id}/questions'

    # Base question data
    question_data = {
        'question': {
            'question_name': question['question_name'],
            'question_type': question['question_type'],
            'question_text': question['question_text'],
            'points_possible': question.get('points_possible', 1)
            # Do not include 'answers' here
        }
    }

    # Include 'answers' only if they exist and the question is not an essay question
    if 'answers' in question and question['question_type'] != 'essay_question':
        question_data['question']['answers'] = question['answers']

    # Include 'general_feedback' if it exists (for essay questions)
    if 'general_feedback' in question:
        question_data['question']['general_feedback'] = question['general_feedback']

    # Send the request to create the question
    response = requests.post(create_question_url, headers=headers, json=question_data)

    if response.status_code == 201 or response.status_code == 200:
        print(f"Added question '{question['question_name']}' successfully.")
    else:
        print(f"Failed to add question '{question['question_name']}'")
        print(f"Status Code: {response.status_code}")
        print(response.text)